## 영화별 줄거리 내  키워드 추출<br>
줄거리 기준 : 네이버 영화 페이지의 줄거리

In [4]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from urllib.request import urlopen

import re
from konlpy.tag import Okt, Kkma, Komoran
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences

import networkx
from tqdm.autonotebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from gensim.models import word2vec
#from difflib import ndiff

import os
os.environ['JAVA_OPTS'] = 'Xmx8192m'

In [5]:
full_execute = False

In [115]:
df = pd.read_csv(('./dataset/책영화리스트_8183.csv'))

In [103]:
keyword_df=keyword_df.set_index(['idx'])

In [116]:
len(df)

8187

In [105]:
keyword_df

,keywords
idx,
0,"[ㄴ, 의, 하, 을, 작품, 하, 카프카, 에, 를, ,, 수, 는, 어, ., ..."
1,"[ㄴ, 어, 의, 에, 부, 을, 세계, 맨, 상, 크, 를, 고, 며, 게, 하,..."
2,"[에, 프랑스, 어, 이, 소설, 을, 의, 있, ㄴ, 는, 하, 베스트셀러, 이,..."
4,"[ㄴ, 을, 는, 감정, 하, 의, 이, 하, 이, ㄹ, 가, 에, 도, 를, 소년..."
6,"[ㄴ, 하, 의, 이, 는, 에, 을, 로, ., 가, 어, 를, 아, 에서, 으로..."
...,...
10000,"[ㄴ, 의, 과, 하, 을, 그녀, 삶, 이야기, 는, 자신]"
10001,"[ㄴ, 정글, 기, 아, 선교사, 대장암, ., 이, 의, 말, 로]"
10002,"[는, ㄴ, 을, ,, 있, 함께, 고, 나아가, 사랑으로, 치유, 용서, 하, 와..."


In [106]:
kdf=pd.concat(['df','keyword_df'],axis=1)

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [101]:
idx=df.index 

In [107]:
kdf=pd.merge(df,keyword_df,left_index=True, right_index=True)

In [110]:
kdf.to_csv('./dataset/책영화리스트_키워드.csv')

In [86]:
len(df)

8183

전처리했으면 패스

In [8]:
df.drop(df[df['plot'].isnull()].index, inplace=True)

df['plot'] = df['plot'].apply(str)
plots = df['plot'].copy()#pd.Series(df['plot'])

df['plot'] = list(map(lambda x:re.sub("\‘[가-힣 ]+\’(\([가-힣 ]+\))?[^\s]*"," ",x),df['plot']))
df['plot'] = list(map(lambda x:re.sub("[가-힣]+ [가-힣]+ {0,1}\([가-힣 ]+\)[^\s]*"," ",x), df['plot']))

for idx in tqdm(df.index):
    chars1, chars2 = [],[]
    
    chars1 = list(set([s[1:-1] for s in re.findall("\‘[가-힣 ]+\’", plots[idx])]))
    if chars1:
        if len(chars1) == 1:
            regex1 = ''.join(list(set([s[1:-1] for s in re.findall("\‘[가-힣 ]+\’", plots[idx])])))
        else:
            regex1 = '|'.join(list(set([s[1:-1] for s in re.findall("\‘[가-힣 ]+\’", plots[idx])])))
        #print(regex1)
        plots[idx] = re.sub('(%s)[^\s]'%(regex1),' ',df.loc[idx,'plot'])

    chars2 = ''.join(re.findall('[가-힣]+ [가-힣]+ {0,1}\([가-힣 ]+\)',plots[idx]))
    if chars2:
        regex2 = '|'.join(map(str.strip, re.compile('\([가-힣 ]+\)').split(chars2)[:-1]))
        regex2 += '|'+' '.join(regex2.split('|')).replace(' ','|')
        #print(regex2)
        plots[idx] = re.sub('(%s)[^\s]'%(regex2),' ',df.loc[idx,'plot'])

df['plot'] = plots

  0%|          | 0/10006 [00:00<?, ?it/s]

In [9]:
del plots

In [10]:
display(len(df['plot']))
df['plot'].head(10)

10006

0    밀란 쿤데라는 카프카의 작품을 두고 '검은색의 기이한 아름다움'이라 표현했다. <변...
1    “샐리 루니는 밀레니얼 세대의 샐린저다”1991년생 맨부커상 후보의 탄생!2020년...
2    당신의 가슴을 행복으로 채워줄 감동의 사랑을 만난다! 출간 2주 만에 프랑스 베스트...
3    당신의 가슴을 행복으로 채워줄 감동의 사랑을 만난다! 출간 2주 만에 프랑스 베스트...
4    괴물인 내가 또 다른 괴물을 만났다!영화와도 같은 강렬한 사건과 매혹적인 문체로 시...
5    괴물인 내가 또 다른 괴물을 만났다!영화와도 같은 강렬한 사건과 매혹적인 문체로 시...
6    정교한 살인수식에 도전하는 천재 물리학자의 집요한 추적이 시작된다!히가시노 게이고 ...
7    비극의 아름다움이 모여 만든 감동의 대서사시이자 토마스 하디가 자신의 대표작으로 꼽...
8    독일 고전주의 문학의 정수로 꼽히는 <파우스트>를 소개하는 책. <파우스트>는 괴테...
9    독일 고전주의 문학의 정수로 꼽히는 <파우스트>를 소개하는 책. <파우스트>는 괴테...
Name: plot, dtype: object

In [11]:
df['plot'] = df['plot'].str.replace('\r\xa0', ' ')
display(df[df['plot'].str.contains('\r')].index)
df['plot'] = df['plot'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 .][^\s]+"," ")

Int64Index([ 15,  16,  51,  52,  78,  79,  80,  81,  82,  83, 121, 122, 151,
            152, 163, 164, 167, 188, 189, 190, 214, 245, 252, 289, 322, 323,
            346, 347, 351, 352, 422, 461, 462, 522, 523, 528, 532, 579, 580,
            607, 627, 629, 655, 723, 734, 767, 812, 814],
           dtype='int64')

/var/folders/pm/x15q7lz11ks31sdtpqsy5z1w0000gn/T/ipykernel_945/2615922936.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['plot'] = df['plot'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 .][^\s]+"," ")


In [12]:
df['plot'].head(20)

0     밀란 쿤데라는 카프카의 작품을 두고   기이한 아름다움  표현했다.   쿤데라의 이...
1       루니는 밀레니얼 세대의 샐린저다  맨부커상 후보의 탄생    영국   드라마 방...
2     당신의 가슴을 행복으로 채워줄 감동의 사랑을 만난다! 출간   만에 프랑스 베스트셀...
3     당신의 가슴을 행복으로 채워줄 감동의 사랑을 만난다! 출간   만에 프랑스 베스트셀...
4     괴물인 내가 또 다른 괴물을 만났다  같은 강렬한 사건과 매혹적인 문체로 시선을 사...
5     괴물인 내가 또 다른 괴물을 만났다  같은 강렬한 사건과 매혹적인 문체로 시선을 사...
6     정교한 살인수식에 도전하는 천재 물리학자의 집요한 추적이 시작된다  게이고 문학의 ...
7     비극의 아름다움이 모여 만든 감동의 대서사시이자 토마스 하디가 자신의 대표작으로 꼽...
8     독일 고전주의 문학의 정수로 꼽히는   소개하는 책.   괴테가   집필을 시작해 ...
9     독일 고전주의 문학의 정수로 꼽히는   소개하는 책.   괴테가   집필을 시작해 ...
10    영화   헤이즐’ 원작 베스트셀러  우리 별에 있어〉 하드커버 에디션 출간말기 암 ...
11    김영하 등단   맞이해 새롭게 선보이는   김영하 소설  네번째 작품으로   기억법...
12    김영하 등단   맞이해 새롭게 선보이는   김영하 소설  네번째 작품으로   기억법...
13    놀라운 통찰력으로   심연을 치밀하게 그리다  작품마다 압도적인 서사와 폭발적인 이...
14    놀라운 통찰력으로   심연을 치밀하게 그리다  작품마다 압도적인 서사와 폭발적인 이...
15      언제까지나 잊지 마, 내가 여기 있었다는 걸 기억해 줘.”\r  월드의 빛나는 ...
16      언제까지나 잊지 마, 내가 여기 있었다는 걸 기억해 줘.”\r  월드의 빛나는 ...
17      함께 조지 오웰을 대표하는 작품으로, 전제주의라는 거대한 지배 시스템 앞

In [26]:
df=df.drop_duplicates(['newtitle'])

In [18]:
del df['Unnamed: 0']

In [27]:
df

,newtitle,plot
0,b변신,밀란 쿤데라는 카프카의 작품을 두고 기이한 아름다움 표현했다. 쿤데라의 이...
1,b노멀 피플,루니는 밀레니얼 세대의 샐린저다 맨부커상 후보의 탄생 영국 드라마 방...
2,b구해줘,당신의 가슴을 행복으로 채워줄 감동의 사랑을 만난다! 출간 만에 프랑스 베스트셀...
4,b아몬드,괴물인 내가 또 다른 괴물을 만났다 같은 강렬한 사건과 매혹적인 문체로 시선을 사...
6,b용의자 X의 헌신,정교한 살인수식에 도전하는 천재 물리학자의 집요한 추적이 시작된다 게이고 문학의 ...
...,...,...
10000,m제인 구달,2 여 년 전 돌연 자신의 모든 업적과 개인적인 삶을 포기하고 지구 환경보호에 뛰어...
10001,m파파 오랑후탄,말레이시아에서 2 년째 사역 중인 박철현 선교사. 효과적인 선교를 위하여 정글 속에...
10002,m기도의 힘,성공 가도를 달리고 있는 남편 사랑스러운 딸 함께 남부럽지 않은 삶을 살고 ...
10004,m폴란드로 간 아이들,한국전쟁 고아 명이 비밀리에 폴란드로 보내졌다. 폴란드 선생님들은 말도 통하...


In [29]:
df.to_csv('책영화리스트_전처리.csv',encoding='utf-8')

In [30]:
okt, kkma, kmr = Okt(), Kkma(), Komoran()

In [31]:
komoran=Komoran()

In [32]:
df.drop_duplicates

<bound method DataFrame.drop_duplicates of           newtitle                                               plot
0              b변신  밀란 쿤데라는 카프카의 작품을 두고   기이한 아름다움  표현했다.   쿤데라의 이...
1           b노멀 피플    루니는 밀레니얼 세대의 샐린저다  맨부커상 후보의 탄생    영국   드라마 방...
2             b구해줘  당신의 가슴을 행복으로 채워줄 감동의 사랑을 만난다! 출간   만에 프랑스 베스트셀...
4             b아몬드  괴물인 내가 또 다른 괴물을 만났다  같은 강렬한 사건과 매혹적인 문체로 시선을 사...
6       b용의자 X의 헌신  정교한 살인수식에 도전하는 천재 물리학자의 집요한 추적이 시작된다  게이고 문학의 ...
...            ...                                                ...
10000       m제인 구달  2 여 년 전 돌연 자신의 모든 업적과 개인적인 삶을 포기하고 지구 환경보호에 뛰어...
10001     m파파 오랑후탄  말레이시아에서 2 년째 사역 중인 박철현 선교사. 효과적인 선교를 위하여 정글 속에...
10002       m기도의 힘  성공 가도를 달리고 있는 남편   사랑스러운 딸   함께 남부럽지 않은 삶을 살고 ...
10004  m폴란드로 간 아이들    한국전쟁 고아   명이 비밀리에 폴란드로 보내졌다. 폴란드 선생님들은 말도 통하...
10005     m리우 2096    인디언 투피남바족이 사는 섬, 신에게서 특별한 능력을 부여받은 전사 아베구아는 ...

[8183 rows x 2 columns]>

In [60]:
df.to_csv('./dataset/책영화리스트.csv')

In [33]:
new.to_csv('./dataset/movie_plot.csv')

### 단어 출현 빈도

In [152]:
from collections import Counter

In [153]:
cnt = Counter()

for plot in tqdm(df['plot']):
    for word,tag in kmr.pos(plot):
        if tag in ['NNP', 'NNG', 'VV']:
            if not cnt[(word,tag)]:
                cnt[(word,tag)] = 1
            else:
                cnt[(word,tag)] += 1

  0%|          | 0/8187 [00:00<?, ?it/s]

AssertionError: phrase input should be string, not <class 'float'>

In [154]:
cnt

Counter({('밀란 쿤데라', 'NNP'): 3,
         ('카프카', 'NNP'): 9,
         ('작품', 'NNG'): 126,
         ('두', 'VV'): 6,
         ('표현', 'NNG'): 23,
         ('쿤', 'NNP'): 2,
         ('데라', 'NNG'): 2,
         ('현대인', 'NNP'): 2,
         ('삶', 'NNG'): 49,
         ('출구', 'NNG'): 1,
         ('찾', 'VV'): 14,
         ('속', 'NNG'): 66,
         ('인간', 'NNG'): 87,
         ('주어지', 'VV'): 4,
         ('불안', 'NNG'): 10,
         ('의식', 'NNG'): 7,
         ('구원', 'NNG'): 4,
         ('꿈', 'NNG'): 16,
         ('군더더기', 'NNG'): 1,
         ('언어', 'NNG'): 20,
         ('형상화', 'NNP'): 3,
         ('발표', 'NNG'): 31,
         ('우리나라', 'NNP'): 5,
         ('소개', 'NNG'): 13,
         ('지도', 'NNG'): 1,
         ('반백', 'NNP'): 1,
         ('넘', 'VV'): 2,
         ('번역본', 'NNG'): 4,
         ('이번', 'NNG'): 16,
         ('출간', 'NNG'): 41,
         ('삽화가', 'NNP'): 2,
         ('돋보이', 'VV'): 4,
         ('아르헨티나', 'NNP'): 1,
         ('아티스트', 'NNP'): 1,
         ('루이스', 'NNP'): 1,
         ('스카파티', 'NNP'): 1,
   

In [35]:
cnt.most_common()

[(('되', 'VV'), 14555),
 (('하', 'VV'), 7491),
 (('있', 'VV'), 7398),
 (('위하', 'VV'), 5486),
 (('자신', 'NNG'), 5425),
 (('시작', 'NNG'), 4480),
 (('사람', 'NNG'), 3650),
 (('사랑', 'NNG'), 3605),
 (('받', 'VV'), 3375),
 (('알', 'VV'), 3074),
 (('만나', 'VV'), 2707),
 (('살', 'VV'), 2593),
 (('날', 'NNG'), 2564),
 (('속', 'NNG'), 2479),
 (('찾', 'VV'), 2240),
 (('사건', 'NNG'), 2156),
 (('보', 'VV'), 2059),
 (('친구', 'NNG'), 2011),
 (('대하', 'VV'), 2007),
 (('가', 'VV'), 1986),
 (('집', 'NNG'), 1929),
 (('일', 'NNG'), 1763),
 (('빠지', 'VV'), 1694),
 (('남자', 'NNG'), 1671),
 (('후', 'NNG'), 1602),
 (('세계', 'NNG'), 1581),
 (('말', 'NNG'), 1581),
 (('삶', 'NNG'), 1555),
 (('떠나', 'VV'), 1523),
 (('세상', 'NNG'), 1503),
 (('가족', 'NNG'), 1482),
 (('오', 'VV'), 1441),
 (('아버지', 'NNG'), 1413),
 (('이야기', 'NNG'), 1357),
 (('여자', 'NNG'), 1331),
 (('마음', 'NNG'), 1312),
 (('느끼', 'VV'), 1267),
 (('때', 'NNG'), 1255),
 (('발견', 'NNG'), 1253),
 (('사실', 'NNG'), 1248),
 (('마을', 'NNG'), 1237),
 (('만들', 'VV'), 1235),
 (('시간', 'NNG'), 1235),


In [155]:
cnt_df = pd.DataFrame(np.hstack(np.array(cnt.most_common())[:,0]).reshape(-1,2),
                     columns=['word','tag'])
cnt_df['cnt'] = np.array(cnt.most_common())[:,1]
cnt_df.head()

/var/folders/pm/x15q7lz11ks31sdtpqsy5z1w0000gn/T/ipykernel_945/1435080750.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cnt_df = pd.DataFrame(np.hstack(np.array(cnt.most_common())[:,0]).reshape(-1,2),
/var/folders/pm/x15q7lz11ks31sdtpqsy5z1w0000gn/T/ipykernel_945/1435080750.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cnt_df['cnt'] = np.array(cnt.most_common())[:,1]


,word,tag,cnt
0,작품,NNG,126
1,작가,NNG,120
2,하,VV,116
3,있,VV,113
4,소설,NNG,105


### 불용어 제거

In [157]:
from nltk.corpus import stopwords
if full_execute:
    stopwords.words('korean')

In [158]:
stopwords

<WordListCorpusReader in '/Users/camilleyoon/nltk_data/corpora/stopwords'>

In [159]:
idx_cnt=50

In [160]:
if full_execute:
    idx_cnt += 50
    cnt_df[cnt_df['tag'].str.startswith('N')][idx_cnt:idx_cnt+50]

#### 수동

In [40]:
'''
cnt_idx = 1300, cnt < 66
'''
ridx = [4, 6, 10, 15, 25, 41, 45, 60, 61, 64, 78, 90, 91, 96, 97, 102, 107, 109, 140, 145, 160, 161, 162, 186, 188,
205, 221, 234, 249, 253, 260, 268, 271, 272, 280, 281, 282, 284, 292, 302, 304, 314, 316, 320, 324, 328, 356,
362, 379, 381, 399, 415, 416, 430, 434, 453, 462, 465, 466, 469, 479, 487, 495, 496, 498, 503, 508, 514, 516,
521, 525, 528, 551, 552, 555, 563, 569, 573, 576, 577, 583, 585, 599, 600, 603, 608, 610, 612, 618, 629, 631,
650, 655, 671, 674, 683, 690, 693, 700, 701, 714, 724, 728, 729, 732, 733, 739, 742, 745, 748, 750, 754, 756,
759, 764, 771, 778, 785, 788, 789, 795, 803, 826, 830, 843, 859, 866, 876, 881, 883, 901, 913, 931, 934, 939,
947, 956, 961, 965, 967, 972, 973, 978, 990, 1010, 1027, 1034, 1045, 1046, 1053, 1058, 1063, 1078, 1088, 1089, 1109, 1110, 1112, 1116, 1120, 1137, 1141, 1145, 1147, 1155, 1157, 1158, 1160, 1168, 1179, 1187, 1207,
1210, 1211, 1214, 1223, 1225, 1231, 1234, 1238, 1241, 1256, 1264, 1267, 1269, 1278, 1283, 1286, 1290, 1293,
1301, 1308, 1316, 1319, 1322, 1341, 1352, 1354, 1355, 1370, 1375, 1390, 1391, 1402, 1415, 1417, 1418, 1421,
1423, 1430, 1432, 1437, 1445, 1456, 1457, 1458, 1459, 1481, 1492, 1497, 1500, 1501, 1504, 1505, 1509, 1517,
1518, 1523, 1533, 1535, 1538, 1541, 1560, 1565, 1575, 1576, 1591, 1595, 1596, 1601, 1607, 1608, 1611, 1617,
1619, 1624, 1634, 1637, 1638, 1642, 1652, 1657, 1663, 1664, 1689, 1699, 1701, 1706, 1721]


#### 수동 이후 부분 중 명사, 길이 = 1

In [272]:
cnt_df

,word,tag,cnt
0,작품,NNG,126
1,작가,NNG,120
2,하,VV,116
3,있,VV,113
4,소설,NNG,105
...,...,...,...
2921,획,NNG,1
2922,지난날,NNG,1
2923,모서리,NNP,1
2924,쓰다듬,VV,1


In [162]:
'''
idx > 1720, tag == N, word len = 1, cnt < 66,
'''
stops_idx = cnt_df[(cnt_df['tag'].str.startswith('N')) & (cnt_df['cnt'] < 66) & (cnt_df['word'].agg(len) == 1)].index

In [163]:
print(cnt_df.loc[stops_idx][['word','tag']].index)
print(cnt_df.loc[ridx,][['word','tag']].index)

Int64Index([  15,   22,   39,   48,   51,   52,   55,   60,   73,   78,
            ...
            2774, 2785, 2796, 2814, 2836, 2837, 2851, 2886, 2887, 2921],
           dtype='int64', length=207)
Int64Index([   4,    6,   10,   15,   25,   41,   45,   60,   61,   64,
            ...
            1642, 1652, 1657, 1663, 1664, 1689, 1699, 1701, 1706, 1721],
           dtype='int64', length=259)


In [164]:
stops = np.concatenate((cnt_df.loc[stops_idx][['word','tag']].values, cnt_df.loc[ridx,][['word','tag']].values))

In [182]:

komoran.pos('의')

[('의', 'NNG')]

In [179]:
komoran.pos(res)

AssertionError: phrase input should be string, not <class 'list'>

In [173]:
cnt_df

,word,tag,cnt
0,작품,NNG,126
1,작가,NNG,120
2,하,VV,116
3,있,VV,113
4,소설,NNG,105
...,...,...,...
2921,획,NNG,1
2922,지난날,NNG,1
2923,모서리,NNP,1
2924,쓰다듬,VV,1


In [165]:
stops

array([['삶', 'NNG'],
       ['말', 'NNG'],
       ['르', 'NNG'],
       ['눈', 'NNG'],
       ['성', 'NNG'],
       ['피', 'NNG'],
       ['앞', 'NNG'],
       ['글', 'NNG'],
       ['후', 'NNG'],
       ['꿈', 'NNG'],
       ['일', 'NNG'],
       ['상', 'NNG'],
       ['공', 'NNG'],
       ['때', 'NNG'],
       ['쿠', 'NNG'],
       ['생', 'NNG'],
       ['이', 'NNP'],
       ['딸', 'NNG'],
       ['안', 'NNG'],
       ['부', 'NNG'],
       ['날', 'NNG'],
       ['시', 'NNG'],
       ['판', 'NNG'],
       ['화', 'NNG'],
       ['코', 'NNG'],
       ['을', 'NNG'],
       ['병', 'NNG'],
       ['귀', 'NNG'],
       ['선', 'NNG'],
       ['실', 'NNG'],
       ['미', 'NNP'],
       ['곳', 'NNG'],
       ['당', 'NNG'],
       ['길', 'NNG'],
       ['방', 'NNG'],
       ['맛', 'NNG'],
       ['윤', 'NNP'],
       ['재', 'NNG'],
       ['최', 'NNP'],
       ['레', 'NNP'],
       ['전', 'NNG'],
       ['로', 'NNG'],
       ['위', 'NNG'],
       ['배', 'NNG'],
       ['폴', 'NNP'],
       ['좀', 'NNG'],
       ['다', 'NNG'],
       ['한', 

In [193]:
stop_pos=['NNP','NNG','VV']

In [197]:
res=[]
for i in ks[:30]:
    o=[]
    for word in i:
        if word not in stops and len(word)>1:
            if komoran.pos(word)=='NNP' or komoran.pos(word)=='NNG' or komoran.pos(word)=='VV':
                o.append(word)
    res.append(o)

In [205]:
komoran.pos('ㄴ다')=='EC'

False

In [198]:
res

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [199]:
ks[:30]

0     [ㄴ, 의, 을, 로, 하, 는, 들, 용의자, 를, ,, 이, 하, 에서, 기, ...
1     [을, ㄴ, 하, 의, ,, 이, 는, 었, ., 에, ㄹ, 이, 다, 는, 어, ...
2                  [2, 에, 어, 들, 할머니, 단, 는, 피해자, 중국, 자락]
3             [매몰, /, 1, 구조, 완료, 으로, 드릴, 광부, 들, 지상, 작업]
4            [의, 들, 을, 이, ㄴ, 를, 하, 아, 위하, 하, ㄴ다, 용사, ,]
5     [하, 를, ㄴ, 이, 아, 전, 세계, 에이전트, 을, 블랙, 는, 위하, 에, ...
6             [ㄴ, 어, ,, 에, 을, 제, 를, 이, 되, 들, 하, 일본군, 중]
7     [ㄴ, 는, 이, 을, 의, 는, 이, 하, 마이크, 은, 들, 에, 지, 그, 고...
8          [의, ㄴ, 이, 을, 영국군, 에, 명령, 를, 사투, 과, 며, 중지, .]
9     [을, 하, ㄴ, 에, ,, 를, 사건, ., 이, 고, 사망, 경찰, 은, 것, ...
10         [하, 의, 들, ㄴ, 에, 고, 이, ,, 을, 멸망, 되, 어, ., ㄴ다]
11    [ㄴ, 에, ,, 을, 이삿짐, 기, 하, 는, 박태호, 를, 은, 고, 아, 3,...
12    [는, ㄴ, 고, 와, 하, 가, 는, ,, 에, 민아, 영재, 이, 를, 이, 에...
13    [ㄴ, 을, 에, 의, 이, 되, 에서, 아, 것, 채, 알, 고립, ,, 게, ....
14                    [아, 기, ㄴ, 통하, 를, 게, 되, 필요, 벚꽃, 하]
15    [ㄴ, 하, ,, 이, 에, 되, 남성, 는, 을, ., 의, 그리고, 들, 이어지...
16             [는, 에, 다, 미스터리, 마지막, 발명품, ,, 유령, 없, 탐지기]
17    [ㄴ, 으로, 새롭, 하, 를, 적, 과, 가, 이, 의, 로, 게, 어, 

In [241]:
len(res)

8183

### 불용어

In [44]:
with open('./stopwords/1.txt',encoding='utf-8') as f:
    stops1 = f.readlines()
with open('./stopwords/2.txt',encoding='utf-8') as f:
    stops2 = f.readlines()
with open('./stopwords/3.txt',encoding='utf-8') as f:
    stops3 = f.readlines()

FileNotFoundError: [Errno 2] No such file or directory: './stopwords/1.txt'

In [40]:
stops1 = list(map(lambda x: x.replace('\n',''),stops1))
stops2 = list(map(lambda x: x.replace('\n',''),stops2))
stops3 = list(map(lambda x: x.replace('\n',''),stops3))

NameError: name 'stops1' is not defined

In [29]:
stopwords = list(set(stops1) & set(stops2) & set(stops3))

### Word2Vec

In [209]:
kkma.pos('ㄴ')

[('ㄴ', 'NNG')]

In [208]:
for plot in tqdm(ks[:3]):
    print(plot)

  0%|          | 0/3 [00:00<?, ?it/s]

['ㄴ', '의', '을', '로', '하', '는', '들', '용의자', '를', ',', '이', '하', '에서', '기', '며', '되', '아', '영애', '경찰', '함께', '고', '둘러싸', '.', '가', '죽음', '지만', '한', '에', '지목', 'ㄴ다']
['을', 'ㄴ', '하', '의', ',', '이', '는', '었', '.', '에', 'ㄹ', '이', '다', '는', '어', '팀', '들', '아', '하', '있', 'ㄴ다', '았', '블랙잭', '것', 'ㄴ데', '빠지', '던', '은', '하', '고', '수', '마다', '만', '교수', '그']
['2', '에', '어', '들', '할머니', '단', '는', '피해자', '중국', '자락']


In [210]:
result_kkma = []
for plot in tqdm(ks[:3]):
    for word in plot:
        tag = kkma.pos(word)
        if tag in ['NNG', 'NNP', 'VV']:
            print(word)
#             words.append(word)
#     words = (' '.join(words)).strip()
#     result_kkma.append(words)

  0%|          | 0/3 [00:00<?, ?it/s]

In [273]:
result_kkma

[]

In [226]:
komoran.pos('ㄴ데')

[('ㄴ데', 'EC')]

In [236]:
no=['ㄴ데','에게','까지','라는','ㄹ러']

In [237]:
res=[]
for plot in ks:
    w=[]
    for word in plot:
        if word not in stops and len(word)>1 and word not in no:
            tag = kkma.pos(word)[0][1]
            if tag in ['NNG', 'NNP', 'VV']:
                w.append(word)
    res.append(w)

In [238]:
len(res)

8183

In [239]:
res[100:120]

[['정훈', '범인', '교실', '사건', '사건', '살인', '발견', '시간'],
 ['수영', '대회', '연습', '동생', '들려주'],
 ['서울', '보스', '친구', '중간', '시절', '도끼'],
 ['생활', '동네', '우즈', '대학'],
 ['로즈', '로티', '위하', '자신', '생기', '생활'],
 ['비밀', '공포', '자신'],
 ['아리엘', '가지', '뉴욕', '는데', '얼마', 'ㄴ지'],
 ['광주', '항쟁', '도시'],
 ['행복', '치료', '인생'],
 ['사랑', '여자'],
 ['지금'],
 [],
 ['기지', '공개', '비밀', '의혹'],
 ['낚이', '작전', '국민', '물귀신', '공블리'],
 ['가방', '상무'],
 ['사랑은'],
 ['사랑'],
 ['오사카', '주장', '달리', '팀워크', '흔들리'],
 ['친구', '옥상', '평소'],
 ['효정']]

#### Kkma

In [45]:
stopwords=['@','#']

In [46]:
if full_execute:
    result_kkma = []
    for plot in tqdm(plots):
        words = []
        for word,tag in kkma.pos(plot):
            if tag in ['NNG', 'NNP', 'VV']:
                words.append(word)
        words = (' '.join(words)).strip()
        result_kkma.append(words)

In [47]:
if full_execute:
    with open('naver_plot_kkma.nlp','w',encoding='utf-8') as f:
        f.write('\n'.join(result_kkma))

    w2v_kkma = word2vec.LineSentence('./model/naver_plot_kkma.nlp')
    model_kkma = word2vec.Word2Vec(w2v_kkma, size=200, window=5, hs=1, min_count=2, sg=1)
    model_kkma.save('./model/naver_plot_kkma.model')

#### Komoran

In [48]:
result_kmr = []
for plot in tqdm(df['plot']):
    words = []
    for word,tag in komoran.pos(plot):
        #if tag in ['NNG', 'NNP', 'VV']:
        if tag in ['NNG', 'NNP']:
            words.append(word)
    words = (' '.join(words)).strip()
    result_kmr.append(words)

  0%|          | 0/8183 [00:00<?, ?it/s]

In [49]:

with open('./model/naver_plot_kmr.nlp','w', encoding='utf-8') as f:
    f.write('\n'.join(result_kmr))

w2v_kmr = word2vec.LineSentence('./model/naver_plot_kmr.nlp')
# model_kmr = word2vec.Word2Vec(w2v_kmr, size=200, window=5, hs=1, min_count=10, sg=1)

model_kmr = word2vec.Word2Vec(w2v_kmr, window=5, hs=1, min_count=10, sg=1)
model_kmr.save('./model/naver_plot_kmr_min10_N_only.model')

#### load & test model

In [50]:
if full_execute:
    model_kkma = word2vec.Word2Vec.load('./model/naver_plot_kkma.model')
model_kmr = word2vec.Word2Vec.load('./model/naver_plot_kmr_min10_N_only.model')

In [51]:
if full_execute:
    word = '레이싱'
    print('most similar words to',word,'in word2vec')
    print('Kkma\t\t\t\tKomoran')
    for (w1,t1),(w2,t2) in zip(model_kkma.wv.most_similar(word,topn=20), model_kmr.wv.most_similar(word,topn=20)):
        print("%-10s\t%.4f\t\t%-10s\t%.4f"%(w1,t1,w2,t2))

### Textrank

In [52]:
#출처 : https://bab2min.tistory.com/570
class RawTaggerReader:
    def __init__(self, plot, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.plot = plot
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        ch = self.rgxSplitter.split(self.plot)
        if len(ch) == 1: ch.append('.')
        for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
            if not s: continue
            yield self.tagger.pos(s)

class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0

    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])

    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()

        if int(len(ranks) * ratio) > 10:
            cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        else:
            cand = sorted(ranks, key=ranks.get, reverse=True)[:10]
        pairness = {}
        startOf = {}
        tuples = {}

        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)

        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if len(k) == 1:
                if used.intersection(set(k)): continue
                both[k] = tuples[k]
                for w in k:
                    used.add(w)
        
        return both

### 키워드 추출

In [53]:
stops=[]

In [54]:
stopwords=['""']

In [55]:
results = []
for idx in tqdm(df.index):
    keywords = []
    tr = TextRank(window=5, coef=1)
    stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV'), ('없', 'VV'),
                       ('위하', 'VV')]).union(set(map(lambda x: tuple(x),stops)))
#     tr.load(RawTaggerReader(df.loc[idx,'plot']),
#             lambda w:w not in stopword and w[0] not in stopwords and
#             w[0] in model_kmr.wv.vocab and (w[1] in ('NNG', 'NNP')))
                #and w[0] in model_kmr.wv.vocab and (w[1] in ('NNG', 'NNP', 'VV')))
    tr.load(RawTaggerReader(df.loc[idx,'plot']))
    tr.build()
    kw = tr.extract(0.2)
    for k in sorted(kw, key=kw.get, reverse=True):
        keywords.append(k[0][0])
            #print("%s\t%g" % (k, kw[k]))
    if idx %100 == 0:
        print(idx,'/10255',keywords)
    results.append(keywords)

  0%|          | 0/8183 [00:00<?, ?it/s]

0 /10255 ['ㄴ', '의', '하', '을', '작품', '하', '카프카', '에', '를', ',', '수', '는', '어', '.', '고', '이', '되', '다', '는', '들', '표현', '은', '이', '은', '으로', '도', '없이', '소개', '아', '삶', '을', '로', '있', 'ㄹ', '아름답']
200 /10255 ['ㄴ', '의', '어', '하', '되', '이', '에', '을', '아', '는', '모습', '이']
400 /10255 ['ㄴ', '의', '을', '에서', '으로', '이', '되', '들', '소년', '작품', '.', '이']
500 /10255 ['ㄴ', ',', '하', '을', '속', '의', '등', '으로', '아픔', '하', '고', '.', '았', '에', '다']
600 /10255 ['ㄴ', '과', '의', '을', '들', '하', '하', '로', '필치', '.']
700 /10255 ['ㄴ', '을', '의', '하', '하', '에', '문인수', '고', '는', '도', '있', '.', '풍경', ',', '으로', '는', '를', '시집', '아', '과거', '이', '다']
800 /10255 ['의', '을', 'ㄴ', '과', '는', '는', '아이들', '를', '으로', '에', '자신', '고', '에게', '함께', '행복', '가족', '하', '느끼', '박목월', '아들', '아버지']
1000 /10255 [',', 'ㄴ', '를', '아빠', '의', '소녀', '게', '되', '엄마', '당하']
1100 /10255 ['ㄴ', '고', '들', '이', '었', '이', '의', '작', '는', '.']
1200 /10255 ['ㄴ', '의', '을', '하', '돈', '이', '고', '시작', '업계', '절대', '빅딜']
1300 /10255 ['을', '의', '는', 'ㄴ', '들', '고', '

7200 /10255 ['의', '을', 'ㄴ', '에', '고', '들', ',', '악마', '이', '은', '하', '게', '.', '분', '세일럼', '어', '과', '마을', '하', 'ㄴ다', '되', '그', '이', '에서', '소녀', '는', '프록터', '부인', '는', '으로', '에']
7300 /10255 [',', '어', '는', 'ㄴ', '에게', '하', '이', '게', '고', '있', '을', '하', '퇴임', '는', '실력', '그', '오', '어']
7400 /10255 ['는', '아', '을', '밀림', '으로', 'ㄴ', '의', '를', '은', '있', '고', '위하', '하']
7500 /10255 ['을', 'ㄴ', '하', '의', '고', '지크프리트', '에', '를', '터', ',', '군', '것', '아', '그', '이', '는', 'ㄴ다', '에게', 'ㄹ', '.', '가', '기', '는', '어', '되', '알', '왕', '하', '게', '자신', 'ㄴ', '지만', '로', '으로', '은', '있', '브룬힐트를', '이', '다는', '걸', '았', '그녀', '라는', '던', '사랑', '와', '들어가', '었', '게', '찾아가', '하겐', '마법', '톨', '보물', '브룬힐트는', '사실', '전설', '용', '에서', '아버지', '후']
7600 /10255 ['를', '을', 'ㄴ', '의', '은', '고', '기', '베토벤', '.', '가족', 'ㄴ다', '하', '에', '미시', '되', '이', '이', '아', '들', '게', '녀']
7700 /10255 ['ㄹ', '의', ',', '를', '는', '음원', 'ㄴ', '을', '하', '그', '모르', '어', '던', '었']
7800 /10255 ['ㄴ', ',', '을', '하', '대', '십', '섹스', '되', '들', '시작', '에', '.']


In [63]:
keyword_df = pd.DataFrame([(idx,result) for idx,result in zip(df.index,results)],columns=['idx','keywords'])

In [64]:
keyword_df

,idx,keywords
0,0,"[ㄴ, 의, 하, 을, 작품, 하, 카프카, 에, 를, ,, 수, 는, 어, ., ..."
1,1,"[ㄴ, 어, 의, 에, 부, 을, 세계, 맨, 상, 크, 를, 고, 며, 게, 하,..."
2,2,"[에, 프랑스, 어, 이, 소설, 을, 의, 있, ㄴ, 는, 하, 베스트셀러, 이,..."
3,4,"[ㄴ, 을, 는, 감정, 하, 의, 이, 하, 이, ㄹ, 가, 에, 도, 를, 소년..."
4,6,"[ㄴ, 하, 의, 이, 는, 에, 을, 로, ., 가, 어, 를, 아, 에서, 으로..."
...,...,...
8178,10000,"[ㄴ, 의, 과, 하, 을, 그녀, 삶, 이야기, 는, 자신]"
8179,10001,"[ㄴ, 정글, 기, 아, 선교사, 대장암, ., 이, 의, 말, 로]"
8180,10002,"[는, ㄴ, 을, ,, 있, 함께, 고, 나아가, 사랑으로, 치유, 용서, 하, 와..."
8181,10004,"[을, ㄴ, 는, 도, ,, 들, 아이들, 폴란드, 은, 선생님, 의, 아, 하, ..."


In [58]:
keyword_df = pd.DataFrame([(idx,result) for idx,result in zip(df.index,results)],columns=['idx','keywords'])

In [59]:
keyword_df

,idx,keywords
0,0,"[ㄴ, 의, 하, 을, 작품, 하, 카프카, 에, 를, ,, 수, 는, 어, ., ..."
1,1,"[ㄴ, 어, 의, 에, 부, 을, 세계, 맨, 상, 크, 를, 고, 며, 게, 하,..."
2,2,"[에, 프랑스, 어, 이, 소설, 을, 의, 있, ㄴ, 는, 하, 베스트셀러, 이,..."
3,4,"[ㄴ, 을, 는, 감정, 하, 의, 이, 하, 이, ㄹ, 가, 에, 도, 를, 소년..."
4,6,"[ㄴ, 하, 의, 이, 는, 에, 을, 로, ., 가, 어, 를, 아, 에서, 으로..."
...,...,...
8178,10000,"[ㄴ, 의, 과, 하, 을, 그녀, 삶, 이야기, 는, 자신]"
8179,10001,"[ㄴ, 정글, 기, 아, 선교사, 대장암, ., 이, 의, 말, 로]"
8180,10002,"[는, ㄴ, 을, ,, 있, 함께, 고, 나아가, 사랑으로, 치유, 용서, 하, 와..."
8181,10004,"[을, ㄴ, 는, 도, ,, 들, 아이들, 폴란드, 은, 선생님, 의, 아, 하, ..."


In [65]:
keyword_df.to_csv('./dataset/all_keywords_0329.csv',index=False, encoding='utf-8')

In [66]:
len(keyword_df)

8183

In [62]:
del keyword_df

In [141]:
keyword_df = pd.read_csv('./movie_keywords_1231.csv')

In [67]:
keyword_df.head(10)

,idx,keywords
0,0,"[ㄴ, 의, 하, 을, 작품, 하, 카프카, 에, 를, ,, 수, 는, 어, ., ..."
1,1,"[ㄴ, 어, 의, 에, 부, 을, 세계, 맨, 상, 크, 를, 고, 며, 게, 하,..."
2,2,"[에, 프랑스, 어, 이, 소설, 을, 의, 있, ㄴ, 는, 하, 베스트셀러, 이,..."
3,4,"[ㄴ, 을, 는, 감정, 하, 의, 이, 하, 이, ㄹ, 가, 에, 도, 를, 소년..."
4,6,"[ㄴ, 하, 의, 이, 는, 에, 을, 로, ., 가, 어, 를, 아, 에서, 으로..."
5,7,"[ㄴ, 의, 을, 적, 이, 하, ,, 는, 에, 기, 사회, 게, 이, 은, .,..."
6,8,"[ㄴ, 하, 의, 을, 는, 과, 에, 이, ,, 를, 가, 어, 파우스트, 하, ..."
7,10,"[을, 고, 에, 는, 하, ㄴ, 눈, 이, 헤이즐, 있, 암, 은, ., 의, 받..."
8,11,"[는, 아, 을, 기억, 를, 의, 하, ㄴ, 에서, 은, 로, 들, 지, 되, 기..."
9,13,"[ㄴ, 을, 의, 하, 적, 이, 어, 는, 하, 정유정, 게, 작품, 으로, 그,..."


In [71]:
keyword_df.shape

(8183, 2)

In [79]:
df.to_csv('./dataset/책영화리스트_8183.csv')

In [84]:
df=pd.read_csv('./dataset/책영화리스트_8183.csv')

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [74]:
kdf=pd.concat([df,keyword_df],axis=1)

In [75]:
kdf

,newtitle,plot,idx,keywords
0,b변신,밀란 쿤데라는 카프카의 작품을 두고 기이한 아름다움 표현했다. 쿤데라의 이...,0.0,"[ㄴ, 의, 하, 을, 작품, 하, 카프카, 에, 를, ,, 수, 는, 어, ., ..."
1,b노멀 피플,루니는 밀레니얼 세대의 샐린저다 맨부커상 후보의 탄생 영국 드라마 방...,1.0,"[ㄴ, 어, 의, 에, 부, 을, 세계, 맨, 상, 크, 를, 고, 며, 게, 하,..."
2,b구해줘,당신의 가슴을 행복으로 채워줄 감동의 사랑을 만난다! 출간 만에 프랑스 베스트셀...,2.0,"[에, 프랑스, 어, 이, 소설, 을, 의, 있, ㄴ, 는, 하, 베스트셀러, 이,..."
4,b아몬드,괴물인 내가 또 다른 괴물을 만났다 같은 강렬한 사건과 매혹적인 문체로 시선을 사...,6.0,"[ㄴ, 하, 의, 이, 는, 에, 을, 로, ., 가, 어, 를, 아, 에서, 으로..."
6,b용의자 X의 헌신,정교한 살인수식에 도전하는 천재 물리학자의 집요한 추적이 시작된다 게이고 문학의 ...,8.0,"[ㄴ, 하, 의, 을, 는, 과, 에, 이, ,, 를, 가, 어, 파우스트, 하, ..."
...,...,...,...,...
8088,NaN,NaN,9905.0,"[돈, 에, 을, ㄴ, 하, 고, 않, 지, 어, 던, 으로, 가, 부부, 남자, ..."
8112,NaN,NaN,9931.0,"[ㄴ, 을, 의, 를, 아, 이, 에, ., ,, 로, 은, 레지나, 고, 었, 조..."
8115,NaN,NaN,9934.0,"[ㄴ, 을, 하, 았, 를, 고, 에, ., 아버지, 있, 하, 그녀, 의, 는, ..."
8143,NaN,NaN,9962.0,"[ㄴ, ,, 크리스마스, 는, 는, 를, 기, 을, 은, 의, 에, 들, 하, 도,..."


In [92]:
df.columns=['idx','title','plot','content']

In [93]:
df

,idx,title,plot,content
0,0,스파이더맨: 노 웨이 홈,계략으로 세상에 정체가 탄로난 스파이더맨 하루 아침에 평범한 일상을 잃게 된...,m
1,1,아네트,예술가들의 도시 오페라 가수 꼬띠아르 스탠드업 코미디언 드라이버 첫눈...,m
2,3,학교 가는 길,전국 특수학교 재학생의 절반은 매일 왕복 거리를 통학하며 전쟁 같은 아침을 맞이...,m
3,4,"쿠오바디스, 아이다",세르비아군이 마을을 공격하자 보스니아 사람들은 안전지역인 캠프로 피신한다. ...,m
4,5,자산어보,양반은 대역 죄인이니 너무 잘해줄 생각들 말어” 순조 신유박해로 세상의 끝 ...,m
...,...,...,...,...
5626,72,소문,모든 것이 뒤바뀐다 미스터리 역사상 최고의 반전이라는 입소문이 자체로 ...,b
5627,73,1026,"남기택, 오홍진, 김현정, 김화선, 오연희",b
5628,74,인생,"매혈기 작가, 위화의 대표작 칸 영화제 심사위원 대상 수상작 원작 소설....",b
5629,75,2020 제11회 젊은작가상 수상작품집,내일을 상상케 하는 눈부신 터닝 포인트!\r \r 등단 이하의 젊은 작가들이...,b


In [91]:
keyword_df

,idx,keywords
0,0,"[가, ㄴ, 을, 하, 아, 에, 게, 불청객, 의, ㄴ다]"
1,1,"[에, 에게, 하, 가수, 꼬, 띠, 아, 르, 함께, ㄴ다]"
2,2,"[을, 의, 하, 에, ㄴ, 는, 를, 학교, 이, 들, 은, 타이, 박, 고, 는..."
3,3,"[하, 캠프, 들, 자, 을, 으로, 이, 는, 아이다, ㄴ다]"
4,4,"[을, 하, 고, ', 에, 의, 아, 알, 이, 어, 다는, ㄴ, 로, 되, 게,..."
...,...,...
5620,5620,"[ㄴ, 의, 하, 한, 는, 을, 에, 어, 아, 하, 기, 경민, ,, 하, 도,..."
5621,5621,"[ㄴ, 을, 꿈, 하, 페니, 는, 고, 로, ,, 에서, 를, 달러, 이, 구, ..."
5622,5622,"[ㄴ, 을, 는, 이, 의, 작품, 어, 적, 』, 들, 으로, 하, 고, 작가, ..."
5623,5623,"[ㄴ, 의, 이, 에서, ,, 여섯, 하, 에, 를, 가, 하, 어, 희생자, 되,..."


In [111]:
merged = pd.merge(df,keyword_df,left_on=df.idx,right_on=keyword_df.idx)

AttributeError: 'DataFrame' object has no attribute 'idx'

In [98]:
merged.drop(columns=['key_0', 'idx_x','idx_y'], inplace=True)

In [99]:
merged

,title,plot,content,keywords
0,스파이더맨: 노 웨이 홈,계략으로 세상에 정체가 탄로난 스파이더맨 하루 아침에 평범한 일상을 잃게 된...,m,"[가, ㄴ, 을, 하, 아, 에, 게, 불청객, 의, ㄴ다]"
1,페스트,극한의 절망 속에서도 억누를 수 없는 희망의 의지 실존주의 문학의 대표 작가 알베...,b,"[가, ㄴ, 을, 하, 아, 에, 게, 불청객, 의, ㄴ다]"
2,아네트,예술가들의 도시 오페라 가수 꼬띠아르 스탠드업 코미디언 드라이버 첫눈...,m,"[에, 에게, 하, 가수, 꼬, 띠, 아, 르, 함께, ㄴ다]"
3,우리가 빛의 속도로 갈 수 없다면,"지난겨울까지 바이오센서를 만드는 과학도였던 김초엽 작가는, 이제 소설을 쓴다. 어디...",b,"[에, 에게, 하, 가수, 꼬, 띠, 아, 르, 함께, ㄴ다]"
4,학교 가는 길,전국 특수학교 재학생의 절반은 매일 왕복 거리를 통학하며 전쟁 같은 아침을 맞이...,m,"[하, 캠프, 들, 자, 을, 으로, 이, 는, 아이다, ㄴ다]"
...,...,...,...,...
5251,1984,"함께 조지 오웰을 대표하는 작품으로, 전제주의라는 거대한 지배 시스템 앞에 놓인...",b,"[ㄴ, 을, 에, 하, 의, 이, ㄹ, 폭발, 백두산, 아, 남과 북, 작전, ,,..."
5252,설국,일본 최초의 노벨문학상 수상작품. 그러나 가와바타 야스나리의 무슨무슨상으로 소개...,b,"[에, ,, 고, ㄴ, 은, 게, 하, 에서, 서로, 규, 의, 태, 는, 는, 으..."
5253,나의 라임 오렌지나무,전 세계인을 울리고 웃긴 다섯 살 소년 제제의 이야기 꼬마 악동 제제의 슬프고 아...,b,"[ㄴ, 하, 의, 는, 길, 시키, 막막, 어, ., 가족]"
5254,28,리얼리티 속에 숨겨진 상징과 생존을 향한 뜨거운 갈망” \n 세계청소년문학...,b,"[을, 는, 의, ㄴ, 이, 하, 사건, 의뢰인, 으로, ,, 는, 폭행, 바로, ..."


In [100]:
merged.to_csv('./merged_all.csv',index=False)

In [117]:
cdf=pd.read_csv('./dataset/cluster_data.csv')

In [120]:
cdf.head(3)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,keywords,plot_preprocessed_kmr,plot_preprocessed_kkma,content
0,19,드라마,2009.0,11.12,1.0,1,한 소녀의 죽음을 둘러싸고 용의자로 몰린 세 명의 열 아홉 청춘들의 이야기. 그녀가...,T.O.P|승리|허이재,김영호|이영범,"['영애', '시작', '지목', '이야기', '살인', '덩어리', '집', '정...",소녀 죽음 용의자 청춘 이야기 한강 익사체 발견 살인 사건 발생 동갑내기 용의자 지...,소녀 죽음 용의자 세 명의 아홉 청춘 이야기 한강 익사체 발견 살인 사건 발생 동갑...,m
1,21,범죄|드라마,2008.0,6.19,3.0,27,"{“위너, 위너, 치킨 디너! (승자, 승자, 닭고기 저녁)”. 그 말이 밤새 머리...",짐 스터게스|케이트 보스워스|아론 유|리자 라피라|제이콥 피츠|조시 게드|샘 골자리...,잭 맥기|헬렌 캐리,"['팀', '카지노', '교수', '입학', '저녁', '승자', '위장', '말'...",위너 위너 치킨 디너 승자 승자 닭고기 저녁 말 밤새 머리 속 베가스 전통 왕년 카...,위너 위너 치킨 디너 승자 승자 닭고기 저녁 말 밤새 머리 속 베 가스 전통 왕년 ...,m
2,22,다큐멘터리,2018.0,8.14,3.5,1,중국 지역 일본군 ‘위안부’ 피해자 2 만 명 2 14년 촬영 당시 생존자 단 22...,박차순|린아이란|리메이진|리아이롄,NaN,"['피해자', '촬영', '삶', '끝', '자락', '지역', '일본군', '중국...",중국 지역 일본군 피해자 촬영 당시 생존자 할머니 마지막 이야기 중국 피해자 할머니...,중국 지역 일본군 피해자 촬영 당시 생존자 할머니 마지막 이야기 현재 중국 피해자 ...,m


In [124]:
cdf.shape

(8927, 13)

In [122]:
keyword_df

,keywords
idx,
0,"[ㄴ, 의, 하, 을, 작품, 하, 카프카, 에, 를, ,, 수, 는, 어, ., ..."
1,"[ㄴ, 어, 의, 에, 부, 을, 세계, 맨, 상, 크, 를, 고, 며, 게, 하,..."
2,"[에, 프랑스, 어, 이, 소설, 을, 의, 있, ㄴ, 는, 하, 베스트셀러, 이,..."
4,"[ㄴ, 을, 는, 감정, 하, 의, 이, 하, 이, ㄹ, 가, 에, 도, 를, 소년..."
6,"[ㄴ, 하, 의, 이, 는, 에, 을, 로, ., 가, 어, 를, 아, 에서, 으로..."
...,...
10000,"[ㄴ, 의, 과, 하, 을, 그녀, 삶, 이야기, 는, 자신]"
10001,"[ㄴ, 정글, 기, 아, 선교사, 대장암, ., 이, 의, 말, 로]"
10002,"[는, ㄴ, 을, ,, 있, 함께, 고, 나아가, 사랑으로, 치유, 용서, 하, 와..."


In [123]:
kdf

,newtitle,plot,keywords
0,b변신,밀란 쿤데라는 카프카의 작품을 두고 기이한 아름다움 표현했다. 쿤데라의 이...,"[ㄴ, 의, 하, 을, 작품, 하, 카프카, 에, 를, ,, 수, 는, 어, ., ..."
1,b노멀 피플,루니는 밀레니얼 세대의 샐린저다 맨부커상 후보의 탄생 영국 드라마 방...,"[ㄴ, 어, 의, 에, 부, 을, 세계, 맨, 상, 크, 를, 고, 며, 게, 하,..."
2,b구해줘,당신의 가슴을 행복으로 채워줄 감동의 사랑을 만난다! 출간 만에 프랑스 베스트셀...,"[에, 프랑스, 어, 이, 소설, 을, 의, 있, ㄴ, 는, 하, 베스트셀러, 이,..."
4,b아몬드,괴물인 내가 또 다른 괴물을 만났다 같은 강렬한 사건과 매혹적인 문체로 시선을 사...,"[ㄴ, 을, 는, 감정, 하, 의, 이, 하, 이, ㄹ, 가, 에, 도, 를, 소년..."
6,b용의자 X의 헌신,정교한 살인수식에 도전하는 천재 물리학자의 집요한 추적이 시작된다 게이고 문학의 ...,"[ㄴ, 하, 의, 이, 는, 에, 을, 로, ., 가, 어, 를, 아, 에서, 으로..."
...,...,...,...
10000,m제인 구달,2 여 년 전 돌연 자신의 모든 업적과 개인적인 삶을 포기하고 지구 환경보호에 뛰어...,"[ㄴ, 의, 과, 하, 을, 그녀, 삶, 이야기, 는, 자신]"
10001,m파파 오랑후탄,말레이시아에서 2 년째 사역 중인 박철현 선교사. 효과적인 선교를 위하여 정글 속에...,"[ㄴ, 정글, 기, 아, 선교사, 대장암, ., 이, 의, 말, 로]"
10002,m기도의 힘,성공 가도를 달리고 있는 남편 사랑스러운 딸 함께 남부럽지 않은 삶을 살고 ...,"[는, ㄴ, 을, ,, 있, 함께, 고, 나아가, 사랑으로, 치유, 용서, 하, 와..."
10004,m폴란드로 간 아이들,한국전쟁 고아 명이 비밀리에 폴란드로 보내졌다. 폴란드 선생님들은 말도 통하...,"[을, ㄴ, 는, 도, ,, 들, 아이들, 폴란드, 은, 선생님, 의, 아, 하, ..."


In [125]:
nt =[]
for i in range(len(cdf)):
    t =str(cdf['title'][i])
    if cdf['content'][i]=='m':
        t='m'+t
    nt.append(t)

In [126]:
len(nt)

8927

In [127]:
cdf['title']=nt

In [128]:
cdf

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,keywords,plot_preprocessed_kmr,plot_preprocessed_kkma,content
0,m19,드라마,2009.0,11.12,1.0,1,한 소녀의 죽음을 둘러싸고 용의자로 몰린 세 명의 열 아홉 청춘들의 이야기. 그녀가...,T.O.P|승리|허이재,김영호|이영범,"['영애', '시작', '지목', '이야기', '살인', '덩어리', '집', '정...",소녀 죽음 용의자 청춘 이야기 한강 익사체 발견 살인 사건 발생 동갑내기 용의자 지...,소녀 죽음 용의자 세 명의 아홉 청춘 이야기 한강 익사체 발견 살인 사건 발생 동갑...,m
1,m21,범죄|드라마,2008.0,6.19,3.0,27,"{“위너, 위너, 치킨 디너! (승자, 승자, 닭고기 저녁)”. 그 말이 밤새 머리...",짐 스터게스|케이트 보스워스|아론 유|리자 라피라|제이콥 피츠|조시 게드|샘 골자리...,잭 맥기|헬렌 캐리,"['팀', '카지노', '교수', '입학', '저녁', '승자', '위장', '말'...",위너 위너 치킨 디너 승자 승자 닭고기 저녁 말 밤새 머리 속 베가스 전통 왕년 카...,위너 위너 치킨 디너 승자 승자 닭고기 저녁 말 밤새 머리 속 베 가스 전통 왕년 ...,m
2,m22,다큐멘터리,2018.0,8.14,3.5,1,중국 지역 일본군 ‘위안부’ 피해자 2 만 명 2 14년 촬영 당시 생존자 단 22...,박차순|린아이란|리메이진|리아이롄,NaN,"['피해자', '촬영', '삶', '끝', '자락', '지역', '일본군', '중국...",중국 지역 일본군 피해자 촬영 당시 생존자 할머니 마지막 이야기 중국 피해자 할머니...,중국 지역 일본군 피해자 촬영 당시 생존자 할머니 마지막 이야기 현재 중국 피해자 ...,m
3,m33,드라마,2016.0,4.07,3.5,1,"2 1 년 8월 5일 칠레 산호세 광산 붕괴 섭씨 32도, 습도 95%의 7 m 지...",안토니오 반데라스|로드리고 산토로|줄리엣 비노쉬,제임스 브롤린|루 다이아몬드 필립스|마리오 카사스|제이콥 바가스|케이트 델 카스틸로...,"['매몰', '완료', '작업', '구조', '칠레', '비상사태', '광부', '...",칠레 산호세 광산 붕괴 섭씨 습도 지하 광부 매몰 매몰 칠레 국가 비상사태 성명 발...,칠 레 산호세 광산 붕괴 섭씨 습도 지하 광 부 매몰 매몰 칠 레 국가 비상사태 성...,m
4,m300,액션|모험|드라마|전쟁,2007.0,3.14,3.4,51,BC 48 년. '크세르크세스' 왕이 이끄는 페르시아 1 만 대군이 그리스를 침공한...,제라드 버틀러|레나 헤디,데이비드 웬햄|도미닉 웨스트|빈센트 레건|마이클 패스벤더|톰 위즈덤|앤드류 프레빈|...,"['침공', '발', '전투', '용맹', '빛', '대군', '용사', '연합',...",왕이 페르시아 만 대군 그리스 침공 그리스군 연합 지연 스파르타의 왕 레오니다스 스...,왕 페르시아 대군 그리스 침공 그리스 군의 연합 지연 스파 르 타의 왕 레 오니 다...,m
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8922,b흙 속에 저 바람 속에,드라마,NaN,NaN,1.5,2,"이 책은 울음, 굶주림, 윷놀이, 돌담, 하얀 옷, '끼리끼리' 등 일상적 소재 속...",이어령,2mwvgYmarxMa7:contents:books:ratings.xlsx,2mwvgYmarxMa7,books,이어령,b
8923,b히틀러 최후의 14일,드라마,NaN,NaN,3.3,2,1945년 4월 16일 20개 대군단 250만 소련 군대가 독일의 수도 베를린 공격...,요아힘 페스트,DgwxAeQYNxrMj:contents:books:ratings.xlsx,DgwxAeQYNxrMj,books,요아힘 페스트,b
8924,bGO,드라마,NaN,NaN,3.6,12,재일동포 3세 고등학생을 주인공으로 한 가네시로 가즈키 자전적 성장소설 『GO』. ...,가네시로 카즈키,2mwvgaAjwxMa7:contents:books:ratings .xlsx,2mwvgaAjwxMa7,books,가네시로 카즈키,b
8925,bY의 비극,드라마,NaN,NaN,4.0,8,세계3대 추리소설 중의 하나인 엘러리 퀸의 소설. 뉴욕 만의 한적한 바다에서 어선에...,엘러리 퀸,2mwvgaAjwxMa7:contents:books:ratings .xlsx,2mwvgaAjwxMa7,books,엘러리 퀸,b


In [129]:
cdf=cdf.drop_duplicates(['title'])

In [131]:
len(kdf)

8183

In [130]:
len(cdf)

8185

In [133]:
merged = pd.merge(cdf,kdf,left_on=cdf.title,right_on=kdf.newtitle)

In [135]:
merged.shape

(8183, 17)

In [136]:
cdf.columns

Index(['title', 'genre', 'year', 'date', 'rating', 'vote_count', 'plot',
       'main_act', 'supp_act', 'keywords', 'plot_preprocessed_kmr',
       'plot_preprocessed_kkma', 'content'],
      dtype='object')

In [139]:
merged.drop(columns=['key_0', 'newtitle','plot_y'], inplace=True)

In [138]:
kdf.head(1)

,newtitle,plot,keywords
0,b변신,밀란 쿤데라는 카프카의 작품을 두고 기이한 아름다움 표현했다. 쿤데라의 이...,"[ㄴ, 의, 하, 을, 작품, 하, 카프카, 에, 를, ,, 수, 는, 어, ., ..."


In [140]:
merged.head(1)

,title,genre,year,date,rating,vote_count,plot_x,main_act,supp_act,keywords_x,plot_preprocessed_kmr,plot_preprocessed_kkma,content,keywords_y
0,m19,드라마,2009.0,11.12,1.0,1,한 소녀의 죽음을 둘러싸고 용의자로 몰린 세 명의 열 아홉 청춘들의 이야기. 그녀가...,T.O.P|승리|허이재,김영호|이영범,"['영애', '시작', '지목', '이야기', '살인', '덩어리', '집', '정...",소녀 죽음 용의자 청춘 이야기 한강 익사체 발견 살인 사건 발생 동갑내기 용의자 지...,소녀 죽음 용의자 세 명의 아홉 청춘 이야기 한강 익사체 발견 살인 사건 발생 동갑...,m,"[ㄴ, 의, 을, 로, 하, 는, 들, 용의자, 를, ,, 이, 하, 에서, 기, ..."


In [144]:
last = merged

In [141]:
kx = merged['keywords_x']
ky=merged['keywords_y']

In [145]:
last['keywords_x']=ky
last['keywords_y']=kx

In [147]:
last.columns=['title', 'genre', 'year', 'date', 'rating', 'vote_count', 'plot_x',
       'main_act', 'supp_act', 'keywords', 'plot_preprocessed_kmr',
       'plot_preprocessed_kkma', 'content', 'keywords_y']

In [148]:
last

,title,genre,year,date,rating,vote_count,plot_x,main_act,supp_act,keywords,plot_preprocessed_kmr,plot_preprocessed_kkma,content,keywords_y
0,m19,드라마,2009.0,11.12,1.0,1,한 소녀의 죽음을 둘러싸고 용의자로 몰린 세 명의 열 아홉 청춘들의 이야기. 그녀가...,T.O.P|승리|허이재,김영호|이영범,"[ㄴ, 의, 을, 로, 하, 는, 들, 용의자, 를, ,, 이, 하, 에서, 기, ...",소녀 죽음 용의자 청춘 이야기 한강 익사체 발견 살인 사건 발생 동갑내기 용의자 지...,소녀 죽음 용의자 세 명의 아홉 청춘 이야기 한강 익사체 발견 살인 사건 발생 동갑...,m,"['영애', '시작', '지목', '이야기', '살인', '덩어리', '집', '정..."
1,m21,범죄|드라마,2008.0,6.19,3.0,27,"{“위너, 위너, 치킨 디너! (승자, 승자, 닭고기 저녁)”. 그 말이 밤새 머리...",짐 스터게스|케이트 보스워스|아론 유|리자 라피라|제이콥 피츠|조시 게드|샘 골자리...,잭 맥기|헬렌 캐리,"[을, ㄴ, 하, 의, ,, 이, 는, 었, ., 에, ㄹ, 이, 다, 는, 어, ...",위너 위너 치킨 디너 승자 승자 닭고기 저녁 말 밤새 머리 속 베가스 전통 왕년 카...,위너 위너 치킨 디너 승자 승자 닭고기 저녁 말 밤새 머리 속 베 가스 전통 왕년 ...,m,"['팀', '카지노', '교수', '입학', '저녁', '승자', '위장', '말'..."
2,m22,다큐멘터리,2018.0,8.14,3.5,1,중국 지역 일본군 ‘위안부’ 피해자 2 만 명 2 14년 촬영 당시 생존자 단 22...,박차순|린아이란|리메이진|리아이롄,NaN,"[2, 에, 어, 들, 할머니, 단, 는, 피해자, 중국, 자락]",중국 지역 일본군 피해자 촬영 당시 생존자 할머니 마지막 이야기 중국 피해자 할머니...,중국 지역 일본군 피해자 촬영 당시 생존자 할머니 마지막 이야기 현재 중국 피해자 ...,m,"['피해자', '촬영', '삶', '끝', '자락', '지역', '일본군', '중국..."
3,m33,드라마,2016.0,4.07,3.5,1,"2 1 년 8월 5일 칠레 산호세 광산 붕괴 섭씨 32도, 습도 95%의 7 m 지...",안토니오 반데라스|로드리고 산토로|줄리엣 비노쉬,제임스 브롤린|루 다이아몬드 필립스|마리오 카사스|제이콥 바가스|케이트 델 카스틸로...,"[매몰, /, 1, 구조, 완료, 으로, 드릴, 광부, 들, 지상, 작업]",칠레 산호세 광산 붕괴 섭씨 습도 지하 광부 매몰 매몰 칠레 국가 비상사태 성명 발...,칠 레 산호세 광산 붕괴 섭씨 습도 지하 광 부 매몰 매몰 칠 레 국가 비상사태 성...,m,"['매몰', '완료', '작업', '구조', '칠레', '비상사태', '광부', '..."
4,m300,액션|모험|드라마|전쟁,2007.0,3.14,3.4,51,BC 48 년. '크세르크세스' 왕이 이끄는 페르시아 1 만 대군이 그리스를 침공한...,제라드 버틀러|레나 헤디,데이비드 웬햄|도미닉 웨스트|빈센트 레건|마이클 패스벤더|톰 위즈덤|앤드류 프레빈|...,"[의, 들, 을, 이, ㄴ, 를, 하, 아, 위하, 하, ㄴ다, 용사, ,]",왕이 페르시아 만 대군 그리스 침공 그리스군 연합 지연 스파르타의 왕 레오니다스 스...,왕 페르시아 대군 그리스 침공 그리스 군의 연합 지연 스파 르 타의 왕 레 오니 다...,m,"['침공', '발', '전투', '용맹', '빛', '대군', '용사', '연합',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8178,b흐르는 강물처럼,드라마,NaN,NaN,3.7,3,꿈꾸던 인생을 살고 있습니까?전세계 1억 독자들의 영혼을 뒤흔든 [연금술사]의 작가...,헨리 데이비드 소로,2mwvgaAjwxMa7:contents:books:ratings .xlsx,"[ㄴ, 의, 하, 을, 들, 에, 를, 에서, 던, 전, ., 출간, 되, 가장, ...",books,헨리 데이비드 소로,b,2mwvgaAjwxMa7
8179,b흙 속에 저 바람 속에,드라마,NaN,NaN,1.5,2,"이 책은 울음, 굶주림, 윷놀이, 돌담, 하얀 옷, '끼리끼리' 등 일상적 소재 속...",이어령,2mwvgYmarxMa7:contents:books:ratings.xlsx,"[,, ㄴ, 적, 의, 을, 이어령, 속, 에세이집, 한국, 이, 다]",books,이어령,b,2mwvgYmarxMa7
8180,b히틀러 최후의 14일,드라마,NaN,NaN,3.3,2,1945년 4월 16일 20개 대군단 250만 소련 군대가 독일의 수도 베를린 공격...,요아힘 페스트,DgwxAeQYNxrMj:contents:books:ratings.xlsx,"[의, 을, 하, ㄴ, ,, 하, 는, 가, 아, 마지막, 들, 히틀러, 게, 생생...",books,요아힘 페스트,b,DgwxAeQYNxrMj
8181,bGO,드라마,NaN,NaN,3.6,12,재일동포 3세 고등학생을 주인공으로 한 가네시로 가즈키 자전적 성장소설 『GO』. ...,가네시로 카즈키,2mwvgaAjwxMa7:contents:books:ratings .xlsx,"[에, 의, 을, ㄴ, 하, 으로, 일본, 적, 는, 되, 에서, 계, 고, 이, ...",books,가네시로 카즈키,b,2mwvgaAjwxMa7


In [149]:
last.to_csv('./dataset/all_keyword.csv')

In [150]:
ks=last['keywords']

In [240]:
ks

0       [ㄴ, 의, 을, 로, 하, 는, 들, 용의자, 를, ,, 이, 하, 에서, 기, ...
1       [을, ㄴ, 하, 의, ,, 이, 는, 었, ., 에, ㄹ, 이, 다, 는, 어, ...
2                    [2, 에, 어, 들, 할머니, 단, 는, 피해자, 중국, 자락]
3               [매몰, /, 1, 구조, 완료, 으로, 드릴, 광부, 들, 지상, 작업]
4              [의, 들, 을, 이, ㄴ, 를, 하, 아, 위하, 하, ㄴ다, 용사, ,]
                              ...                        
8178    [ㄴ, 의, 하, 을, 들, 에, 를, 에서, 던, 전, ., 출간, 되, 가장, ...
8179              [,, ㄴ, 적, 의, 을, 이어령, 속, 에세이집, 한국, 이, 다]
8180    [의, 을, 하, ㄴ, ,, 하, 는, 가, 아, 마지막, 들, 히틀러, 게, 생생...
8181    [에, 의, 을, ㄴ, 하, 으로, 일본, 적, 는, 되, 에서, 계, 고, 이, ...
8182            [ㄴ, 의, 는, 에, 되, 하, 터, 이, 해, 발견, 이, 요크, .]
Name: keywords, Length: 8183, dtype: object

In [267]:
last.tail(5)

,title,genre,year,date,rating,vote_count,plot_x,main_act,supp_act,keywords,plot_preprocessed_kmr,plot_preprocessed_kkma,content,keywords_y
8178,b흐르는 강물처럼,드라마,NaN,NaN,3.7,3,꿈꾸던 인생을 살고 있습니까?전세계 1억 독자들의 영혼을 뒤흔든 [연금술사]의 작가...,헨리 데이비드 소로,2mwvgaAjwxMa7:contents:books:ratings .xlsx,"[출간, 세계, 파울로 코엘료, 풍경, 작가, 로우]",books,헨리 데이비드 소로,b,2mwvgaAjwxMa7
8179,b흙 속에 저 바람 속에,드라마,NaN,NaN,1.5,2,"이 책은 울음, 굶주림, 윷놀이, 돌담, 하얀 옷, '끼리끼리' 등 일상적 소재 속...",이어령,2mwvgYmarxMa7:contents:books:ratings.xlsx,"[이어령, 에세이집, 한국]",books,이어령,b,2mwvgYmarxMa7
8180,b히틀러 최후의 14일,드라마,NaN,NaN,3.3,2,1945년 4월 16일 20개 대군단 250만 소련 군대가 독일의 수도 베를린 공격...,요아힘 페스트,DgwxAeQYNxrMj:contents:books:ratings.xlsx,"[마지막, 히틀러, 순간, 권력]",books,요아힘 페스트,b,DgwxAeQYNxrMj
8181,bGO,드라마,NaN,NaN,3.6,12,재일동포 3세 고등학생을 주인공으로 한 가네시로 가즈키 자전적 성장소설 『GO』. ...,가네시로 카즈키,2mwvgaAjwxMa7:contents:books:ratings .xlsx,"[일본, 면서, 조총련, 주인공, 다니]",books,가네시로 카즈키,b,2mwvgaAjwxMa7
8182,bY의 비극,드라마,NaN,NaN,4.0,8,세계3대 추리소설 중의 하나인 엘러리 퀸의 소설. 뉴욕 만의 한적한 바다에서 어선에...,엘러리 퀸,2mwvgaAjwxMa7:contents:books:ratings .xlsx,"[발견, 요크]",books,엘러리 퀸,b,2mwvgaAjwxMa7


In [266]:
last['keywords']=res

In [271]:
last.to_csv('./dataset/all_keyword_0329.csv')

In [261]:
type(last['keywords_y'][24])

str

In [251]:
last['keywords_y'][15:30]

15                                                  NaN
16    ['후', '흉가', '미스터리', '동아리', '분석', '세상', '귀신', '...
17                                                  NaN
18                                                  NaN
19    ['황제', '훈련', '방어', '의사', '기구', '모양', '옛날', '중국...
20    ['임무', '위기', '인해', '추궁', '잠입', '수행', '사상', '책임...
21    ['연관', '이후', '위기', '절체절명', '사상', '조직', '암호', '...
22    ['확인', '세계', '개발', '위협', '정보', '가게', '베일', '위장...
23                                                  NaN
24    ['조직', '닉', '도미', '인물', '본인', '배신', '남미', '계좌'...
25           ['성욕', '사랑', '남편', '선상', '남자', '운명', '여인']
26    ['세상', '미지', '부족', '미래', '끝', '여정', '노예', '동물'...
27    ['왓슨', '선고', '살아남기', '마니', '군가', '정체', '불명', '...
28    ['후', '집', '성사', '친구', '린지', '공동', '운영', '유로',...
29                                                  NaN
Name: keywords_y, dtype: object

In [269]:
las=[]
for i in range(len(last)):
    if last['content'][i]=='b':
        las.append(last['keywords'][i])
    else:
        if type(last['keywords_y'][i])==str and last['content'][i]=='m':
            las.append(last['keywords_y'][i])
        else:
            las.append(last['keywords'][i])

In [270]:
len(las)

8183